- Register your model
- Create an endpoint and a first deployment
- Deploy a trial run
- Manually send test data to the deployment
- Get details of the deployment
- Create a second deployment
- Manually scale the second deployment
- Update allocation of production traffic between both deployments
- Get details of the second deployment
- Roll out the new deployment and delete the first one

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credentials = DefaultAzureCredential()

# handle to the workspace
ml_client = MLClient(
    credential=credentials,
    subscription_id="a9ea30e6-5728-4375-ac04-cbe4918c3def",
    resource_group_name="rg-training",
    workspace_name="ws-training"
)

In [4]:
# from azure.ai.ml.entities import Model
# from azure.ai.ml.constants import AssetTypes

# mlflow_model = Model(
#     path="./deploy/credit_defaults_model/",
#     type=AssetTypes.MLFLOW_MODEL,
#     name="credit_defaults_model",
#     description="MLflow Model created from local files.",
# )
# ml_client.models.create_or_update(mlflow_model)

In [5]:
registered_model_name = "credit_defaults_model"
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)
print(latest_model_version)

1


# Create an online endpoint

In [6]:
import uuid
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

In [7]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="online endpoint for credit defaults",
    auth_mode="key", #auth_mode : Use key for key-based authentication. Use aml_token for Azure Machine Learning token-based authentication. A key doesn't expire, but aml_token does expire. 
    tags={
        "training-dataset": "credit_defaults"
    }
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [8]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f"Endpoint {endpoint.name} with provisioning state {endpoint.provisioning_state} is retrieved"
)

Endpoint credit-endpoint-c53983ae with provisioning state Succeeded is retrieved


# Deploy the model to the endpoint

In [9]:
from azure.ai.ml.entities import ManagedOnlineDeployment

model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1
)

In [10]:
blue_deployment = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Check: endpoint credit-endpoint-c53983ae exists
Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


.................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/swagger.json', 'name': 'credit-endpoint-c53983ae', 'description': 'online endpoint for credit defaults', 'tags': {'training-dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Ankita Himanshubhai Bhagat (Quadrant Resource LLC)', 'createdAt': '2024-07-07T04:43:12.679704+0000', 'lastModifiedAt': '2024-07-07T04:43:12.679704+0000', 'azureml.onlineendpointid': '/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/resourcegroups/rg-training/providers/microsoft.machinelearningservices/workspaces/ws-training/onlineendpoints/credit-endpoint-c53983ae', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStat

# Check the status of the endpoint

In [11]:
endpoint = ml_client.online_endpoints.get(online_endpoint_name)

print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

Name: credit-endpoint-c53983ae
Status: Succeeded
Description: online endpoint for credit defaults


In [12]:
print(endpoint.traffic)
print(endpoint.scoring_uri)

{'blue': 100}
https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/score


# Test the endpoint with sample data

In [13]:
import os

deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [14]:
%%writefile {deploy_dir}/sample-request.json
{
    "input_data": {
        "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
        "index": [0,1],
        "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
    }
}

Writing ./deploy/sample-request.json


In [15]:
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="./deploy/sample-request.json"
)

'[1, 0]'

# Get logs of the deployment

In [16]:
logs = ml_client.online_deployments.get_logs(
    name="blue", 
    endpoint_name=online_endpoint_name, 
    lines=50
)
print(logs)

Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-07-07T04:51:58.807001Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-07-07T04:52:03.663751Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-07-07T04:52:08.806015Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-07-07T04:52:13.662718Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-07-07T04:52:18.805877Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning

# Create a second deployment

In [17]:
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

green_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1
)

green_deployment = ml_client.online_deployments.begin_create_or_update(
    green_deployment
).result()

Check: endpoint credit-endpoint-c53983ae exists


..............................................................................................

# Scale deployment to handle more traffic

In [18]:
green_deployment.instance_count = 2

ml_client.online_deployments.begin_create_or_update(green_deployment).result()

Check: endpoint credit-endpoint-c53983ae exists


..........................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'credit-endpoint-c53983ae', 'type': 'Managed', 'name': 'green', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/odidp:77c52ac9-a5e2-47ea-a0dc-d01bacc875f6:fd588d73-2b8b-4da4-9058-80e430b7b5c0?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/resourceGroups/rg-training/providers/Microsoft.MachineLearningServices/workspaces/ws-training/onlineEndpoints/credit-endpoint-c53983ae/deployments/green', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/v-ankbhagat2/code/Users/v-ankbhagat/credit-defaults-model', 'creation_context': None, 'serialize': <msrest.serialization

# Update traffic allocation for deployments

In [19]:
endpoint.traffic = {"blue": 80, "green": 20}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/swagger.json', 'name': 'credit-endpoint-c53983ae', 'description': 'online endpoint for credit defaults', 'tags': {'training-dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Ankita Himanshubhai Bhagat (Quadrant Resource LLC)', 'createdAt': '2024-07-07T04:43:12.679704+0000', 'lastModifiedAt': '2024-07-07T04:43:12.679704+0000', 'azureml.onlineendpointid': '/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/resourcegroups/rg-training/providers/microsoft.machinelearningservices/workspaces/ws-training/onlineendpoints/credit-endpoint-c53983ae', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStat

In [21]:
for i in range(30):
    ml_client.online_endpoints.invoke(
        endpoint_name=online_endpoint_name,
        request_file="./deploy/sample-request.json"
    )

In [22]:
logs = ml_client.online_deployments.get_logs(
    name="green",
    endpoint_name=online_endpoint_name,
    lines=50
)
print(logs)

Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-07-07T05:00:53.766857Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-07-07T05:00:58.941999Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-07-07T05:01:03.767108Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-07-07T05:01:08.94208Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-07-07T05:01:13.769404Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, 

# Send all traffic to the new deployment

In [23]:
endpoint.traffic = {"blue": 0, "green": 100}
ml_client.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://credit-endpoint-c53983ae.eastus.inference.ml.azure.com/swagger.json', 'name': 'credit-endpoint-c53983ae', 'description': 'online endpoint for credit defaults', 'tags': {'training-dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Ankita Himanshubhai Bhagat (Quadrant Resource LLC)', 'createdAt': '2024-07-07T04:43:12.679704+0000', 'lastModifiedAt': '2024-07-07T04:43:12.679704+0000', 'azureml.onlineendpointid': '/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/resourcegroups/rg-training/providers/microsoft.machinelearningservices/workspaces/ws-training/onlineendpoints/credit-endpoint-c53983ae', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/a9ea30e6-5728-4375-ac04-cbe4918c3def/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStat

# Delete the old deployment

In [24]:
ml_client.online_deployments.begin_delete(
    name="blue",
    endpoint_name=online_endpoint_name
).result()

# Clean up resources

In [25]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name).result()

.....................................................................................